In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json


# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weatherdata = pd.read_csv("../Output_data/WeatherData-Quarterly_GPMformat.csv")
weatherdata.head()

,CITY,INDICATOR,1951,1952,1953,1954,1955,1956,1957,1958,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,BOISE,Temperature (oC) Q1,1.4,3.3,5.7,6.0,2.0,4.3,6.6,6.0,...,1.8,2.6,4.9,2.9,4.5,0.8,3.6,5.4,4.9,2.3
1,BOISE,Temperature (oC) Q2,16.5,19.8,19.0,20.3,19.0,20.6,19.7,21.3,...,14.5,15.6,13.5,13.0,15.7,15.7,15.3,17.3,17.3,15.2
2,BOISE,Temperature (oC) Q3,17.2,19.6,18.9,17.5,18.2,17.8,18.1,19.1,...,22.9,23.8,22.5,24.1,24.5,24.1,23.8,23.1,21.9,24.0
3,BOISE,Temperature (oC) Q4,-0.9,1.8,2.7,0.1,2.3,-1.0,0.5,2.5,...,6.1,3.8,6.5,5.6,7.0,3.3,6.5,6.1,5.6,4.9
4,BOSTON,Temperature (oC) Q1,2.2,1.2,2.4,1.0,0.4,0.2,0.2,-0.1,...,1.4,-0.2,2.0,0.2,4.1,0.9,-1.1,-3.2,2.6,0.0


### Take mean of temp for all cities for the years 1951 and 2017

In [3]:
weather_data = weatherdata.loc[:, ["CITY", "1951", "2017"]]
weather_data = weather_data.groupby("CITY", as_index=False).mean()

weather_data

,CITY,1951,2017
0,BOISE,8.550,11.600
1,BOSTON,11.275,0.000
2,COLUMBUS,10.775,13.075
3,DES MOINES,8.200,0.000
4,MIAMI,24.175,26.175
5,NASHVILLE,15.125,16.925
6,NEW ORLEANS,8.200,0.000
7,PHOENIX,21.150,25.150
8,SALT LAKE CITY,10.375,13.400
9,SAN DIEGO,16.875,18.975


In [4]:
lonlat = pd.read_csv("../Output_data/LatLon.csv")
#Rename column City to CITY
lonlat.rename(columns = {"City": "CITY"}, inplace = True)
#Change all the rows to upper case in column CITY
lonlat["CITY"] = lonlat["CITY"].str.upper()
lonlat = lonlat.loc[:, ["CITY", "Lat", "Lng"]]
lonlat.head()


,CITY,Lat,Lng
0,BOSTON,42.36,-71.06
1,COLUMBUS,39.96,-83.00
2,MIAMI,25.76,-80.19
3,DES MOINES,41.59,-93.63
4,NASHVILLE,36.17,-86.78


### Merge the two files


In [5]:
weather_data1 = pd.merge(weather_data, lonlat, on="CITY", how= "outer")

weather = weather_data1.dropna()
weather

,CITY,1951,2017,Lat,Lng
0,BOISE,8.550,11.600,43.62,-116.20
1,BOSTON,11.275,0.000,42.36,-71.06
2,COLUMBUS,10.775,13.075,39.96,-83.00
3,DES MOINES,8.200,0.000,41.59,-93.63
4,MIAMI,24.175,26.175,25.76,-80.19
5,NASHVILLE,15.125,16.925,36.17,-86.78
6,NEW ORLEANS,8.200,0.000,29.95,-90.07
7,PHOENIX,21.150,25.150,33.45,-112.07
8,SALT LAKE CITY,10.375,13.400,40.76,-111.89
9,SAN DIEGO,16.875,18.975,32.72,-117.16


### Temperature Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and 1951 as the weight.
* Add Heatmap layer to map.

In [6]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Use the Lat and Lng as locations and Humidity as the weight.
locations = weather[["Lat", "Lng"]]
meantemp_1951 = weather["1951"].astype(float)

# Plot Heatmap
fig = gmaps.figure(map_type="TERRAIN")

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=meantemp_1951, 
                                 dissipating=False, max_intensity=weather["1951"].max(),
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

In [7]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Use the Lat and Lng as locations and Humidity as the weight.
locations = weather[["Lat", "Lng"]]
meantemp_2017 = weather["2017"].astype(float)

# Plot Heatmap
fig = gmaps.figure(map_type="TERRAIN")

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=meantemp_2017, 
                                 dissipating=False, max_intensity=weather["2017"].max(),
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))